# 数据处理

## 1、导入数据

In [0]:
from google.colab import drive
#将Drive中的文件与Colab关联，执行这个命令授权是将Drive挂载到‘/content/drive’下，查看此时的目录就知道了
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os 
path="/content/drive/My Drive/Colab Notebooks/NLP/Project/"
os.chdir(path) #切换当前文件夹

colab是google提供的运行在云端的jupyter notebook环境。这个环境是运行在google虚拟机上的，显然与自己的pc不在一个文件系统。

In [1]:
import pandas as pd
# train=pd.read_json("./dataset/train.txt")
# dev=pd.read_json("./dataset/dev.txt")
# test=pd.read_json("./dataset/test.txt")
train=pd.read_json("/Users/yangshoujian/iCloud云盘（归档）/研一/NLP/55/自然语言处理课程作业/作业二：模型训练/dataset/train.txt")
dev=pd.read_json("/Users/yangshoujian/iCloud云盘（归档）/研一/NLP/55/自然语言处理课程作业/作业二：模型训练/dataset/dev.txt")
test=pd.read_json("/Users/yangshoujian/iCloud云盘（归档）/研一/NLP/55/自然语言处理课程作业/作业二：模型训练/dataset/test.txt")

In [2]:
print(train.values[0])
train.head()

['[0NICH3]dis 预后与其出血类型有关。 [15脑室周围18]bod -脑室内出血的近期预后与出血量大小有关，出血量越大，并发 [47脑积水的发生率或病死率越高59]sym ；远期随访，出血量大者多发生 [74严重智能减退79]sym 和 [81运动功能障碍86]sym 等。 [89[小脑]bod 出血92]sym 预后差，出生后不久即死亡。 [106新生儿 [蛛网膜下腔]bod 出血115]sym 主要系静脉破裂所致， [126出血量较小130]sym ，大多预后良好；少数也可因先天性颅内动脉瘤破裂所致，病情多危重，预后较差，病死率高达40%。 [177幕上硬膜180]bod 下出血预后相对较好，而 [192幕下硬膜195]bod 下出血预后差。'
 {'No.0: 脑积水的发生率或病死率越高': {'has_problem': False, 'subject': {'val': '发生率或病死率越高', 'pos': [51, 59]}, 'self': {'val': '脑积水的发生率或病死率越高', 'pos': [47, 59]}, 'body': {'val': '脑室周围', 'pos': [15, 18]}, 'decorate': {'val': '', 'pos': []}, 'frequency': {'val': '', 'pos': []}, 'item': {'val': '', 'pos': []}, 'disease': {'val': 'ICH', 'pos': [0, 3]}}, 'No.1: 严重智能减退': {'has_problem': False, 'subject': {'val': '智能减退', 'pos': [76, 79]}, 'self': {'val': '严重智能减退', 'pos': [74, 79]}, 'body': {'val': '', 'pos': []}, 'decorate': {'val': '严重', 'pos': [74, 75]}, 'frequency': {'val': '', 'pos': []}, 'item': {'val': '', 'pos': []}, 'disease': {'val': '', 'pos': []}}, 'No.3: 小脑出血': {'h

,text,symptom
0,[0NICH3]dis 预后与其出血类型有关。 [15脑室周围18]bod -脑室内出血的近...,"{'No.0: 脑积水的发生率或病死率越高': {'has_problem': False,..."
1,[0ICH2]dis 的预后与其发病年龄、病因、出血部位及出血量大小等有关。 [30脑动静脉...,"{'No.4: 产生脑积水': {'has_problem': False, 'subjec..."
2,①有明显感染 [6病灶7]sym ，如 [10脐炎11]dis 、 [13肺炎14]dis ...,"{'No.8: 脑脊液鼻漏': {'has_problem': False, 'subjec..."
3,[0颅内压明显增高6]sym 时可用 呋塞米每次1mg/kg 静推，20% 甘露醇每次0.5...,"{'No.0: 颅内压明显增高': {'has_problem': False, 'subj..."
4,治疗过程中脑脊液检查好转，而 [14体温持续不退19]sym ，临床症状不消失；病情好转后又...,"{'No.2: 抽搐': {'has_problem': False, 'subject':..."


## 2、数据格式处理

### 2.1 原始数据清理

清理原始文本中的多余标记

tag=[dis,sym,bod,ite]

类型1：tag去掉，tag后面的半角空格符号去掉

类型2：去掉"["和"]"，以及中间的数字，若"["前有空格，则去掉


In [3]:
# step1: 替换所有tag类型
# step2: 找到所有'['的位置
# step3: 替换"["和"]"，以及删除中间的数字（删除不等于替换，删除针对单个元素）
def clean_tag(text):
    tags=["dis","sym","bod","ite"] #列表
    tag_indexs=[] #定义空列表
    #替换tag
    for t in tags: #遍历列表每个元素
        text=text.replace(t,""*3) #将tag替换成“”
    
    #括号处理
    text_list=list(text) #将任何可迭代数据转换为列表类型,并返回转换后的列表
    right=[i for i,x in enumerate(text_list) if x ==']'] #找到']'的位置
    left=[i for i,x in enumerate(text_list) if x =='[']
    numbers=['0','1','2','3','4','5','6','7','8','9']
    #删除右括号及数字
    if right != -1:
        for i in right: #遍历每个']'
            j=i-1
            while (text_list[j] in numbers): #删除']'及左边的数字
                text_list[j]=" "
                j=j-1
    #删除做左括号及数字
    if left != -1:
        for i in left:
            j=i+1
            while (text_list[j] in numbers):
                text_list[j]=" "
                j=j+1

    #最后替换和整合
    text="".join(text_list) #join()用于将序列中的元素以指定的字符连接生成一个新的字符串
    text=text.replace("]","").replace("[","").replace(" ","")
    return text

In [4]:
s=clean_tag(train.values[0][0])
s[177:181]#'幕上硬膜'

'幕上硬膜'

### 2.2 json处理

In [5]:
#import json
for item in train['symptom'][0]:
    print(item)

No.0: 脑积水的发生率或病死率越高
No.1: 严重智能减退
No.3: 小脑出血
No.5: 出血量较小
No.4: 新生儿蛛网膜下腔出血
No.2: 运动功能障碍


In [6]:
def get_anno_text(line,length):
    #每一条数据的处理
    #line=train['symptom'][0]
    #annotations=['self','subject','body','decorate','frequency','item','disease']
    annotations=['subject','body','decorate','frequency','item','disease']
    items=[item for item in line] #每一条数据下标注的症状，将for循环的结果存储到列表item中，第一个item是满足条件的值。
    #保存每一种症状下，对应标注的位置
    anno_pos=[]
    for anno in annotations: #遍历每个标签
        temp=[]
        for item in items: #每一种症状下的标注
            anno_len=len(line[item][anno]['pos']) #pos[,]数字的个数
            #奇数情况处理
            if anno_len%2==1:
                #anno_len-=1 #减掉一个也会出现问题，可能中间多了一个错误点
                return None #删除这条症状数据
            if anno_len!= 0 and anno_len<4: #单个词语位置
                temp.append(line[item][anno]['pos'][0:anno_len]) #直接取单个词语位置
            elif anno_len!= 0 and anno_len>=4: #多个位置的情况
                i=0
                while i< anno_len:
                    temp.append(line[item][anno]['pos'][i:i+2]) #每次取2个位置
                    i+=2
            
        anno_pos.append(temp)
    #print(anno_pos)

    #生成与原始文本相同长度的标注，并在标注位置上写明症状
    anno_text=list('O'*length) #用全“O”初始化填充列表
    anno_text2=list('O'*length)
    anno_text3=list('O'*length)
    for index in range(len(annotations)): #每一种标注
        if len(anno_pos[index]) != 0:
            #anno_pos[index]=list(set(anno_pos[index]))
            #不对重复的位置进行多次标记
            anno_pos_set=[]
            for i in anno_pos[index]:
                #对于同一个位置上可能出现不同实体相互覆盖的情况，按照实体集合annotations中的先后顺序，后者覆盖前者的策略进行覆盖。
                if i not in anno_pos_set:
                    anno_pos_set.append(i)
            for start , end in anno_pos_set:
                #对start和end之间的字符写入标注
                #print(index,start,end)
                begin=start
                while start < end+1 :
                    if start >= length:
                        #print("wrong data ")
                        return None
                    #@TODO 存在覆盖的情况，因为有的位置可以既是body，又是self
                    if anno_text[start] == 'O':
                        anno_text[start]=annotations[index] #替换'O'
                    else:
                        #检查删除重复情况 
                        if annotations[index] not in anno_text[start]:
                            anno_text[start]=anno_text[start]+" "+annotations[index] #多标签
                    
                    #第二种标注（单标签）
                    anno_text2[start]=annotations[index]

                    #第三种标注（BIO格式）
                    if begin==start: #第一次进入
                        anno_text3[start]="B-"+annotations[index]
                    else:
                        anno_text3[start]="I-"+annotations[index]
                    start+=1
    #print(len(anno_text),len(anno_text2),len(anno_text3))
    return anno_text, anno_text2, anno_text3

In [7]:
#整体数据处理
def data_process(df):
    text_clean=[]
    anno_text=[]
    anno_text2=[]
    anno_text3=[]
    w_indexs=[]
    for i in range(len(df)): #遍历每条数据的索引
        no_tag=clean_tag(df['text'][i])
        text_clean.append(list(no_tag)) #将每个字隔开成列表
        #print(i,len(no_tag))
        if get_anno_text(df['symptom'][i],len(no_tag)) == None: #len(no_tag)每个文本的字数
            w_indexs.append(i)
            anno_text.append(None)
            anno_text2.append(None)
            anno_text3.append(None)
        else:
            anno, anno2, anno3=get_anno_text(df['symptom'][i],len(no_tag))
            anno_text.append(anno)
            anno_text2.append(anno2)
            anno_text3.append(anno3)
    print("sum of wrong data : ", len(w_indexs))
    #奇数34，越界32
    text_clean=list(text_clean)
    print(len(text_clean),len(anno_text2),len(anno_text3))
    result=pd.DataFrame()
    result['text_clean']=text_clean
    result['anno_text']=anno_text
    result['anno_text2']=anno_text2
    result['anno_text3']=anno_text3
    #删除有错误的那行数据
    result=result.drop(w_indexs)
    return result
train_clean=data_process(train)
dev_clean=data_process(dev)
test_clean=data_process(test)
train_clean.to_json("/Users/yangshoujian/iCloud云盘（归档）/研一/NLP/55/自然语言处理课程作业/作业二：模型训练/dataset/train_clean_v3.txt") # 按行转json
dev_clean.to_json("/Users/yangshoujian/iCloud云盘（归档）/研一/NLP/55/自然语言处理课程作业/作业二：模型训练/dataset/dev_clean_v3.txt")
test_clean.to_json("/Users/yangshoujian/iCloud云盘（归档）/研一/NLP/55/自然语言处理课程作业/作业二：模型训练/dataset/test_clean_v3.txt")
train_clean.head()

sum of wrong data :  63
4227 4227 4227
sum of wrong data :  7
318 318 318
sum of wrong data :  37
824 824 824


,text_clean,anno_text,anno_text2,anno_text3
0,"[N, I, C, H, 预, 后, 与, 其, 出, 血, 类, 型, 有, 关, 。, ...","[disease, disease, disease, disease, O, O, O, ...","[disease, disease, disease, disease, O, O, O, ...","[B-disease, I-disease, I-disease, I-disease, O..."
1,"[I, C, H, 的, 预, 后, 与, 其, 发, 病, 年, 龄, 、, 病, 因, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[①, 有, 明, 显, 感, 染, 病, 灶, ，, 如, 脐, 炎, 、, 肺, 炎, ...","[O, O, O, O, O, O, subject, subject, O, O, O, ...","[O, O, O, O, O, O, subject, subject, O, O, O, ...","[O, O, O, O, O, O, B-subject, I-subject, O, O,..."
3,"[颅, 内, 压, 明, 显, 增, 高, 时, 可, 用, 呋, 塞, 米, 每, 次, ...","[item, item, item, decorate, decorate, subject...","[item, item, item, decorate, decorate, subject...","[B-item, I-item, I-item, B-decorate, I-decorat..."
4,"[治, 疗, 过, 程, 中, 脑, 脊, 液, 检, 查, 好, 转, ，, 而, 体, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, ite...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, ite...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-i..."


In [8]:
df1=pd.read_json("/Users/yangshoujian/iCloud云盘（归档）/研一/NLP/55/自然语言处理课程作业/作业二：模型训练/dataset/train_clean_v3.txt")
df2=pd.read_json("/Users/yangshoujian/iCloud云盘（归档）/研一/NLP/55/自然语言处理课程作业/作业二：模型训练/dataset/dev_clean_v3.txt")
df3=pd.read_json("/Users/yangshoujian/iCloud云盘（归档）/研一/NLP/55/自然语言处理课程作业/作业二：模型训练/dataset/test_clean_v3.txt")
df=pd.concat([df1,df2,df3])
df.head()

,text_clean,anno_text,anno_text2,anno_text3
0,"[N, I, C, H, 预, 后, 与, 其, 出, 血, 类, 型, 有, 关, 。, ...","[disease, disease, disease, disease, O, O, O, ...","[disease, disease, disease, disease, O, O, O, ...","[B-disease, I-disease, I-disease, I-disease, O..."
1,"[I, C, H, 的, 预, 后, 与, 其, 发, 病, 年, 龄, 、, 病, 因, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[①, 有, 明, 显, 感, 染, 病, 灶, ，, 如, 脐, 炎, 、, 肺, 炎, ...","[O, O, O, O, O, O, subject, subject, O, O, O, ...","[O, O, O, O, O, O, subject, subject, O, O, O, ...","[O, O, O, O, O, O, B-subject, I-subject, O, O,..."
3,"[颅, 内, 压, 明, 显, 增, 高, 时, 可, 用, 呋, 塞, 米, 每, 次, ...","[item, item, item, decorate, decorate, subject...","[item, item, item, decorate, decorate, subject...","[B-item, I-item, I-item, B-decorate, I-decorat..."
4,"[治, 疗, 过, 程, 中, 脑, 脊, 液, 检, 查, 好, 转, ，, 而, 体, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, ite...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, ite...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-i..."


In [15]:
print(len(df))

5262


In [9]:
l=df['text_clean'][0]
type(l)

pandas.core.series.Series

In [10]:
tags=[]
for anno in df1["anno_text"]:
    #print(anno)
    for i in anno:
        if i not in tags:
            tags.append(i)
print(len(tags))#有多少种标签
tags

24


['disease',
 'O',
 'body',
 'subject',
 'decorate',
 'frequency',
 'subject body',
 'item',
 'subject decorate',
 'body item',
 'subject item',
 'decorate item',
 'body disease',
 'decorate frequency',
 'body decorate',
 'subject disease',
 'subject frequency',
 'frequency disease',
 'decorate disease',
 'subject body disease',
 'item disease',
 'subject body item',
 'subject item disease',
 'body item disease']

In [11]:
tags=[]
for anno in df1["anno_text2"]:
    #print(anno)
    for i in anno:
        if i not in tags:
            tags.append(i)
print(len(tags))#有多少种标签
tags

7


['disease', 'O', 'body', 'subject', 'decorate', 'frequency', 'item']

In [12]:
tags=[]
for anno in df["anno_text3"]:
    #print(anno)
    for i in anno:
        if i not in tags:
            tags.append(i)
print(len(tags))#有多少种标签
tags

13


['B-disease',
 'I-disease',
 'O',
 'B-body',
 'I-body',
 'B-subject',
 'I-subject',
 'B-decorate',
 'I-decorate',
 'B-frequency',
 'I-frequency',
 'B-item',
 'I-item']

In [13]:
df1["anno_text3"][0]

['B-disease',
 'I-disease',
 'I-disease',
 'I-disease',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-body',
 'I-body',
 'I-body',
 'I-body',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-subject',
 'I-subject',
 'I-subject',
 'I-subject',
 'I-subject',
 'I-subject',
 'I-subject',
 'I-subject',
 'I-subject',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-decorate',
 'I-decorate',
 'B-subject',
 'I-subject',
 'I-subject',
 'I-subject',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-subject',
 'I-subject',
 'O',
 'O',
 'B-body',
 'I-body',
 'B-subject',
 'I-subject',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-body',
 'I-body',
 'I-body',
 'I-body',
 'I-body',
 'B-subject',
 'I-subject',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',